In [1]:
import pandas as pd
import datetime as dt
import ast
import geopandas as gpd
import numpy as np
pd.options.display.float_format = "{:.2f}".format

In [14]:
COUNTY = '037'
CITY = 'Nashville'

In [10]:
# county = gpd.read_file('../data/county_cbg.csv')
# county['geoid20'] = county['geoid20'].astype(float)

county  = gpd.read_file('../data/Tennessee Census Block Groups/tl_2020_47_bg.shp')
county = county[county.COUNTYFP == COUNTY]
county.GEOID = county.GEOID.astype(str)

In [13]:
county_cbgs = county.GEOID.unique().tolist()

In [15]:
safe_df =  pd.read_parquet(f'../data/safegraph.parquet/year=2021/region=TN/city={CITY}/', engine='pyarrow')

In [16]:
pd.set_option('display.max_columns', None)
safe_df.head()

,brands,bucketed_dwell_times,category_tags,closed_on,date_begin,date_end,date_range_end,date_range_start,dayofmonth,device_type,distance_from_home,enclosed,geometry_type,includes_parking_lot,is_synthetic,iso_country_code,latitude,location_name,longitude,median_dwell,month,naics_code,normalized_visits_by_region_naics_visitors,normalized_visits_by_region_naics_visits,normalized_visits_by_state_scaling,normalized_visits_by_total_visitors,normalized_visits_by_total_visits,open_hours,opened_on,parent_placekey,phone_number,placekey,poi_cbg,polygon_class,polygon_wkt,postal_code,raw_visit_counts,raw_visitor_counts,related_same_day_brand,related_same_week_brand,safegraph_brand_ids,street_address,sub_category,top_category,tracking_closed_since,visitor_country_of_origin,visitor_daytime_cbgs,visitor_home_aggregation,visitor_home_cbgs,visits_by_day,visits_by_each_hour,visits_friday,visits_monday,visits_saturday,visits_sunday,visits_thursday,visits_tuesday,visits_wednesday,wkt_area_sq_meters
0,nan,"{""<5"":0,""5-10"":6,""11-20"":1,""21-60"":4,""61-120"":...",nan,nan,2021-03-29,2021-04-05,2021-04-05T00:00:00-05:00,2021-03-29T00:00:00-05:00,29,"{""android"":4,""ios"":4}",21136.00,false,POLYGON,false,false,US,36.18,Joe Rutter Nashville Real Estate Pros,-86.77,21.50,3,531210.00,0.00,0.00,245.09,0.00,0.00,nan,nan,nan,16155042017.00,22j-222@5pn-kbn-j7q,470370126001.00,OWNED_POLYGON,POLYGON ((-86.77078560799998 36.18431863800003...,37207.00,14.00,8.00,"{""Shell Oil"":20,""Walmart"":10,""Target"":10,""Barn...","{""Chick-fil-A"":50,""Mapco"":38,""Shell Oil"":38,""S...",nan,801 N 2nd St Unit A,Offices of Real Estate Agents and Brokers,Offices of Real Estate Agents and Brokers,2019-07-01,"{""US"":7}",{},"{""47053966500"":4,""47037011001"":4,""47149040901"":4}","{""470370110011"":4,""471490409011"":4}","[2,2,1,1,1,5,2]","[0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0...",1,2,5,2,1,2,1,210.00
1,nan,"{""<5"":3,""5-10"":22,""11-20"":5,""21-60"":14,""61-120...",Alcoholic Beverages,nan,2021-06-07,2021-06-14,2021-06-14T00:00:00-05:00,2021-06-07T00:00:00-05:00,7,"{""android"":12,""ios"":28}",12352.00,false,POLYGON,false,false,US,36.11,Airport Liquor,-86.67,19.00,6,445310.00,0.00,0.00,961.17,0.00,0.00,nan,nan,nan,16153616891.00,222-222@5pn-kgc-8d9,470370156134.00,OWNED_POLYGON,POLYGON ((-86.67123659899994 36.10507527100003...,37217.00,57.00,43.00,"{""Walmart"":11,""Mapco"":11,""Twice Daily"":8,""McDo...","{""McDonald's"":42,""Shell Oil"":37,""Walmart"":30,""...",nan,1605 Murfreesboro Pike,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores",2019-07-01,"{""US"":41}","{""470370102023"":4,""471251020031"":4,""4718903090...","{""47149040501"":4,""47037015620"":4,""47147080606""...","{""120111103241"":7,""471490417003"":5,""4703701562...","[13,7,7,7,7,7,9]","[0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,5,2,0,1,0,0,0...",7,13,7,9,7,7,7,427.00
2,nan,"{""<5"":0,""5-10"":5,""11-20"":1,""21-60"":1,""61-120"":...",nan,nan,2021-03-22,2021-03-29,2021-03-29T00:00:00-05:00,2021-03-22T00:00:00-05:00,22,"{""android"":4,""ios"":4}",21987.00,false,POLYGON,false,false,US,36.18,Joe Rutter Nashville Real Estate Pros,-86.77,59.50,3,531210.00,0.00,0.00,242.07,0.00,0.00,nan,nan,nan,16155042017.00,22j-222@5pn-kbn-j7q,470370126001.00,OWNED_POLYGON,POLYGON ((-86.77078560799998 36.18431863800003...,37207.00,14.00,7.00,"{""McDonald's"":27,""Dunkin'"":18,""Sonic"":9,""Five ...","{""Sonic"":29,""BP"":29,""Starbucks"":29,""McDonald's...",nan,801 N 2nd St Unit A,Offices of Real Estate Agents and Brokers,Offices of Real Estate Agents and Brokers,2019-07-01,"{""US"":5}","{""291892205022"":4}","{""47037015631"":5,""47037013900"":4,""47037019117"":4}","{""470370139001"":5}","[0,3,1,3,2,3,2]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",2,0,3,2,3,3,1,210.00
3,nan,"{""<5"":3,""5-10"":28,""11-20"":2,""21-60"":13,""61-120...",Alcoholic Beverages,nan,2021-12-20,2021-12-27,2021-12-27T00:00:00-06:00,2021-12-20T00:00:00-06:00,20,"{""android"":17,""ios"":25}",11011.00,false,POLYGON,false,false,US,36.

In [17]:
# reference : # https://docs.safegraph.com/docs/places

safe_df.columns

Index(['brands', 'bucketed_dwell_times', 'category_tags', 'closed_on',
       'date_begin', 'date_end', 'date_range_end', 'date_range_start',
       'dayofmonth', 'device_type', 'distance_from_home', 'enclosed',
       'geometry_type', 'includes_parking_lot', 'is_synthetic',
       'iso_country_code', 'latitude', 'location_name', 'longitude',
       'median_dwell', 'month', 'naics_code',
       'normalized_visits_by_region_naics_visitors',
       'normalized_visits_by_region_naics_visits',
       'normalized_visits_by_state_scaling',
       'normalized_visits_by_total_visitors',
       'normalized_visits_by_total_visits', 'open_hours', 'opened_on',
       'parent_placekey', 'phone_number', 'placekey', 'poi_cbg',
       'polygon_class', 'polygon_wkt', 'postal_code', 'raw_visit_counts',
       'raw_visitor_counts', 'related_same_day_brand',
       'related_same_week_brand', 'safegraph_brand_ids', 'street_address',
       'sub_category', 'top_category', 'tracking_closed_since',
       'vi

In [18]:
# l= ['category_tags', 'naics_code', 'date_begin','distance_from_home', 'includes_parking_lot','location_name', 'poi_cbg','open_hours','raw_visit_counts', 'related_same_day_brand','related_same_week_brand','sub_category', 'top_category','visits_by_day','visits_friday','visits_saturday', 'visits_sunday', 'visits_monday', 'visits_tuesday', 'visits_wednesday', 'visits_thursday', 'polygon_wkt', 'latitude', 'longitude']
l= ['category_tags', 'naics_code', 'date_begin','distance_from_home', 'includes_parking_lot','location_name', 'poi_cbg','open_hours','raw_visit_counts', 'sub_category', 'top_category','visits_by_day','visits_friday','visits_saturday', 'visits_sunday', 'visits_monday', 'visits_tuesday', 'visits_wednesday', 'visits_thursday', 'latitude', 'longitude']

In [33]:
df = safe_df.drop(safe_df.columns.difference(['visitor_home_aggregation', 'visitor_home_cbgs']+ l), 1)
df.head()

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,category_tags,date_begin,distance_from_home,includes_parking_lot,latitude,location_name,longitude,naics_code,open_hours,poi_cbg,raw_visit_counts,sub_category,top_category,visitor_home_aggregation,visitor_home_cbgs,visits_by_day,visits_friday,visits_monday,visits_saturday,visits_sunday,visits_thursday,visits_tuesday,visits_wednesday
0,nan,2021-03-29,21136.00,false,36.18,Joe Rutter Nashville Real Estate Pros,-86.77,531210.00,nan,470370126001.00,14.00,Offices of Real Estate Agents and Brokers,Offices of Real Estate Agents and Brokers,"{""47053966500"":4,""47037011001"":4,""47149040901"":4}","{""470370110011"":4,""471490409011"":4}","[2,2,1,1,1,5,2]",1,2,5,2,1,2,1
1,Alcoholic Beverages,2021-06-07,12352.00,false,36.11,Airport Liquor,-86.67,445310.00,nan,470370156134.00,57.00,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores","{""47149040501"":4,""47037015620"":4,""47147080606""...","{""120111103241"":7,""471490417003"":5,""4703701562...","[13,7,7,7,7,7,9]",7,13,7,9,7,7,7
2,nan,2021-03-22,21987.00,false,36.18,Joe Rutter Nashville Real Estate Pros,-86.77,531210.00,nan,470370126001.00,14.00,Offices of Real Estate Agents and Brokers,Offices of Real Estate Agents and Brokers,"{""47037015631"":5,""47037013900"":4,""47037019117"":4}","{""470370139001"":5}","[0,3,1,3,2,3,2]",2,0,3,2,3,3,1
3,Alcoholic Beverages,2021-12-20,11011.00,false,36.11,Airport Liquor,-86.67,445310.00,nan,470370156134.00,60.00,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores","{""47037015618"":5,""47037019114"":4,""47113001610""...","{""470879603002"":4,""470370156312"":4,""4703701561...","[11,8,12,13,11,0,5]",11,11,0,5,13,8,12
4,nan,2021-12-27,NaN,false,36.18,Joe Rutter Nashville Real Estate Pros,-86.77,531210.00,nan,470370126001.00,3.00,Offices of Real Estate Agents and Brokers,Offices of Real Estate Agents and Brokers,{},"{""470650030002"":4}","[0,0,0,0,0,2,1]",0,0,2,1,0,0,0


In [34]:
df['poi_cbg'] = df['poi_cbg'].apply(lambda x: str(x).split('.')[0])
df = df[df['poi_cbg'].isin(county_cbgs)].reset_index(drop = True)

In [36]:
df['visitor_home_cbgs'] = pd.DataFrame(df['visitor_home_cbgs'].apply(ast.literal_eval))

In [37]:
def func(x):
    return pd.DataFrame(x.items(), columns = ['home_cbg','frequency'])

In [38]:
df['visitor_info'] = df.apply(lambda x: func(x['visitor_home_cbgs']), axis =1)

In [39]:
df.head()

,category_tags,date_begin,distance_from_home,includes_parking_lot,latitude,location_name,longitude,naics_code,open_hours,poi_cbg,raw_visit_counts,sub_category,top_category,visitor_home_aggregation,visitor_home_cbgs,visits_by_day,visits_friday,visits_monday,visits_saturday,visits_sunday,visits_thursday,visits_tuesday,visits_wednesday,visitor_info
0,nan,2021-03-29,21136.00,false,36.18,Joe Rutter Nashville Real Estate Pros,-86.77,531210.00,nan,470370126001,14.00,Offices of Real Estate Agents and Brokers,Offices of Real Estate Agents and Brokers,"{""47053966500"":4,""47037011001"":4,""47149040901"":4}","{'470370110011': 4, '471490409011': 4}","[2,2,1,1,1,5,2]",1,2,5,2,1,2,1,home_cbg frequency 0 470370110011 ...
1,Alcoholic Beverages,2021-06-07,12352.00,false,36.11,Airport Liquor,-86.67,445310.00,nan,470370156134,57.00,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores","{""47149040501"":4,""47037015620"":4,""47147080606""...","{'120111103241': 7, '471490417003': 5, '470370...","[13,7,7,7,7,7,9]",7,13,7,9,7,7,7,home_cbg frequency 0 120111103241 ...
2,nan,2021-03-22,21987.00,false,36.18,Joe Rutter Nashville Real Estate Pros,-86.77,531210.00,nan,470370126001,14.00,Offices of Real Estate Agents and Brokers,Offices of Real Estate Agents and Brokers,"{""47037015631"":5,""47037013900"":4,""47037019117"":4}",{'470370139001': 5},"[0,3,1,3,2,3,2]",2,0,3,2,3,3,1,home_cbg frequency 0 470370139001 ...
3,Alcoholic Beverages,2021-12-20,11011.00,false,36.11,Airport Liquor,-86.67,445310.00,nan,470370156134,60.00,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores","{""47037015618"":5,""47037019114"":4,""47113001610""...","{'470879603002': 4, '470370156312': 4, '470370...","[11,8,12,13,11,0,5]",11,11,0,5,13,8,12,home_cbg frequency 0 470879603002 ...
4,nan,2021-12-27,NaN,false,36.18,Joe Rutter Nashville Real Estate Pros,-86.77,531210.00,nan,470370126001,3.00,Offices of Real Estate Agents and Brokers,Offices of Real Estate Agents and Brokers,{},{'470650030002': 4},"[0,0,0,0,0,2,1]",0,0,2,1,0,0,0,home_cbg frequency 0 470650030002 ...


In [40]:
pd.set_option('display.max_columns', None)
df = df.sort_values(by = ['date_begin']).reset_index(drop = True)
df.head()

,category_tags,date_begin,distance_from_home,includes_parking_lot,latitude,location_name,longitude,naics_code,open_hours,poi_cbg,raw_visit_counts,sub_category,top_category,visitor_home_aggregation,visitor_home_cbgs,visits_by_day,visits_friday,visits_monday,visits_saturday,visits_sunday,visits_thursday,visits_tuesday,visits_wednesday,visitor_info
0,"Country Club,Golf,Golf Lessons,Sports Clubs",2021-01-04,16136.00,true,36.22,Gaylord Springs Golf Links,-86.68,713910.00,"{ ""Mon"": [[""7:00"", ""19:00""]], ""Tue"": [[""7:00"",...",470370153002,26.00,Golf Courses and Country Clubs,Other Amusement and Recreation Industries,"{""47187050905"":4,""47189030308"":4,""47165021008""...","{'471890303081': 6, '470370133004': 4, '471890...","[1,0,8,6,9,1,1]",9,1,1,1,6,0,8,home_cbg frequency 0 471890303081 ...
1,nan,2021-01-04,13519.00,nan,36.14,Firehouse Subs,-86.88,722513.00,"{ ""Mon"": [[""10:30"", ""21:00""]], ""Tue"": [[""10:30...",470370182011,82.00,Limited-Service Restaurants,Restaurants and Other Eating Places,"{""47037018301"":5,""47037015625"":4,""47165021008""...","{'471870501021': 5, '470370132022': 4, '470370...","[11,14,11,22,13,8,3]",13,11,8,3,22,14,11,home_cbg frequency 0 471870501021 ...
2,"Churches,Hindu Temple,Mosque,Sikh Temple,Synag...",2021-01-04,13111.00,false,36.07,Sharing Hearts Joining Hands Corp,-86.73,813110.00,nan,470370189023,9.00,Religious Organizations,Religious Organizations,"{""47037018904"":4,""47037015624"":4}",{'470370103012': 4},"[0,0,1,1,3,2,2]",3,0,2,2,1,0,1,home_cbg frequency 0 470370103012 ...
3,"Jewelry,Women's Clothing",2021-01-04,29834.00,false,36.12,Shubha,-86.75,4481.00,nan,470370173002,5.00,nan,Clothing Stores,"{""47037019115"":4}",{},"[0,0,1,1,1,2,0]",1,0,2,0,1,0,1,"Empty DataFrame Columns: [home_cbg, frequency]..."
4,Churches,2021-01-04,30258.00,false,36.16,Hope Community Church,-86.85,813110.00,nan,470370133003,25.00,Religious Organizations,Religious Organizations,"{""47037010202"":4,""47165020102"":4}","{'471650201022': 4, '470370133004': 4, '471650...","[4,8,0,7,2,3,1]",2,4,3,1,7,8,0,home_cbg frequency 0 471650201022 ...


In [41]:
chunks = np.array_split(df, 5)

In [61]:
chunks

[                                           category_tags  date_begin  \
 0            Country Club,Golf,Golf Lessons,Sports Clubs  2021-01-04   
 1                                                    nan  2021-01-04   
 2      Churches,Hindu Temple,Mosque,Sikh Temple,Synag...  2021-01-04   
 3                               Jewelry,Women's Clothing  2021-01-04   
 4                                               Churches  2021-01-04   
 ...                                                  ...         ...   
 57681                                    Candles,Perfume  2021-03-15   
 57682                                                nan  2021-03-15   
 57683                                           Churches  2021-03-15   
 57684                                 Hardware and Tools  2021-03-15   
 57685                                                nan  2021-03-15   
 
        distance_from_home includes_parking_lot  latitude  \
 0                16136.00                 true     36.22   


In [62]:
t = [chunks[i].groupby('date_begin') for i in range(5)]

In [63]:
x = t[0].get_group(dt.date(2021, 1, 4))

In [67]:
x

,category_tags,date_begin,distance_from_home,includes_parking_lot,latitude,location_name,longitude,naics_code,open_hours,poi_cbg,raw_visit_counts,sub_category,top_category,visitor_home_aggregation,visitor_home_cbgs,visits_by_day,visits_friday,visits_monday,visits_saturday,visits_sunday,visits_thursday,visits_tuesday,visits_wednesday,visitor_info
0,"Country Club,Golf,Golf Lessons,Sports Clubs",2021-01-04,16136.00,true,36.22,Gaylord Springs Golf Links,-86.68,713910.00,"{ ""Mon"": [[""7:00"", ""19:00""]], ""Tue"": [[""7:00"",...",470370153002,26.00,Golf Courses and Country Clubs,Other Amusement and Recreation Industries,"{""47187050905"":4,""47189030308"":4,""47165021008""...","{'471890303081': 6, '470370133004': 4, '471890...","[1,0,8,6,9,1,1]",9,1,1,1,6,0,8,home_cbg frequency 0 471890303081 ...
1,nan,2021-01-04,13519.00,nan,36.14,Firehouse Subs,-86.88,722513.00,"{ ""Mon"": [[""10:30"", ""21:00""]], ""Tue"": [[""10:30...",470370182011,82.00,Limited-Service Restaurants,Restaurants and Other Eating Places,"{""47037018301"":5,""47037015625"":4,""47165021008""...","{'471870501021': 5, '470370132022': 4, '470370...","[11,14,11,22,13,8,3]",13,11,8,3,22,14,11,home_cbg frequency 0 471870501021 ...
2,"Churches,Hindu Temple,Mosque,Sikh Temple,Synag...",2021-01-04,13111.00,false,36.07,Sharing Hearts Joining Hands Corp,-86.73,813110.00,nan,470370189023,9.00,Religious Organizations,Religious Organizations,"{""47037018904"":4,""47037015624"":4}",{'470370103012': 4},"[0,0,1,1,3,2,2]",3,0,2,2,1,0,1,home_cbg frequency 0 470370103012 ...
3,"Jewelry,Women's Clothing",2021-01-04,29834.00,false,36.12,Shubha,-86.75,4481.00,nan,470370173002,5.00,nan,Clothing Stores,"{""47037019115"":4}",{},"[0,0,1,1,1,2,0]",1,0,2,0,1,0,1,"Empty DataFrame Columns: [home_cbg, frequency]..."
4,Churches,2021-01-04,30258.00,false,36.16,Hope Community Church,-86.85,813110.00,nan,470370133003,25.00,Religious Organizations,Religious Organizations,"{""47037010202"":4,""47165020102"":4}","{'471650201022': 4, '470370133004': 4, '471650...","[4,8,0,7,2,3,1]",2,4,3,1,7,8,0,home_cbg frequency 0 471650201022 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5350,"Churches,Hindu Temple,Mosque,Sikh Temple,Synag...",2021-01-04,NaN,false,36.06,Center For Jewish Awareness,-86.92,813110.00,nan,470370184043,1.00,Religious Organizations,Religious Organizations,{},{'470370167004': 4},"[0,0,0,0,1,0,0]",1,0,0,0,0,0,0,home_cbg frequency 0 470370167004 ...
5351,nan,2021-01-04,NaN,false,36.11,Omega Watches,-86.82,423940.00,"{ ""Mon"": [[""11:00"", ""18:00""]], ""Tue"": [[""11:00...",470370179014,2.00,"Jewelry, Watch, Precious Stone, and Precious M...",Miscellaneous Durable Goods Merchant Wholesalers,"{""47165021201"":4}",{},"[0,1,0,0,0,1,0]",0,0,1,0,0,1,0,"Empty DataFrame Columns: [home_cbg, frequency]..."
5352,nan,2021-01-04,26205.00,false,36.20,Motel 6,-86.78,721110.00,"{ ""Mon"": [[""0:00"", ""24:00""]], ""Tue"": [[""0:00"",...",470370127022,14.00,Hotels (except Casino Hotels) and Motels,Traveler Accommodation,"{""01023956700"":4,""47037012702"":4}",{'470370156184': 4},"[8,0,1,0,3,0,2]",3,8,0,2,0,0,1,home_cbg frequency 0 470370156184 ...
5353,Psychics and Astrology,2021-01-04,3219.00,false,36.22,The Psychic Shop,-86.73,812990.00,"{ ""Mon"": [[""9:00"", ""21:00""]], ""Tue"": [[""9:00"",...",470370111002,16.00,All Other Personal Services,Other Personal Services,"{""47037011400"":4}","{'470370112006': 4, '470370114002': 4}","[5,1,0,4,4,2,0]",4,5,2,0,4,1,0,home_cbg frequency 0 470370112006 ...


In [80]:
x.columns

Index(['category_tags', 'date_begin', 'distance_from_home',
       'includes_parking_lot', 'latitude', 'location_name', 'longitude',
       'naics_code', 'open_hours', 'poi_cbg', 'raw_visit_counts',
       'sub_category', 'top_category', 'visitor_home_aggregation',
       'visitor_home_cbgs', 'visits_by_day', 'visits_friday', 'visits_monday',
       'visits_saturday', 'visits_sunday', 'visits_thursday', 'visits_tuesday',
       'visits_wednesday', 'visitor_info'],
      dtype='object')

In [78]:
for keys in t[0].groups:
    print (keys)
# t[0].get_group(key)

2021-01-04
2021-01-11
2021-01-18
2021-01-25
2021-02-01
2021-02-08
2021-02-15
2021-02-22
2021-03-01
2021-03-08
2021-03-15


In [100]:
res = pd.DataFrame()

for key in t[0].groups:
    x = t[0].get_group(key)
    x = x.reset_index(drop = True)
    y = pd.DataFrame()
    for i in range(len(x[:10])):
        temp = x['visitor_info'][i]
        alpha = pd.DataFrame(data=df.loc[i,l]).T
        temp = pd.concat([temp,alpha], axis = 1) 
        temp['date_begin'] = x['date_begin'][i]
        temp['poi_cbg'] = x['poi_cbg'][i]
        temp['category_tags'] = x['category_tags'][i]
        temp['naics_code'] = x['naics_code'][i]
        temp['distance_from_home'] = x['distance_from_home'][i]
        temp['includes_parking_lot'] = x['includes_parking_lot'][i]
        temp['location_name'] = x['location_name'][i]
        temp['open_hours'] = x['open_hours'][i]
        temp['raw_visit_counts'] = x['raw_visit_counts'][i]
        # temp['related_same_day_brand'] = x['related_same_day_brand'][i]
        # temp['related_same_week_brand'] = x['related_same_week_brand'][i]
        temp['sub_category'] = x['sub_category'][i]
        temp['top_category'] = x['top_category'][i]
        temp['visits_by_day'] = x['visits_by_day'][i]
        temp['visits_monday'] = x['visits_monday'][i]
        temp['visits_tuesday'] = x['visits_tuesday'][i]
        temp['visits_wednesday'] = x['visits_wednesday'][i]
        temp['visits_thursday'] = x['visits_thursday'][i]
        temp['visits_friday'] = x['visits_friday'][i]
        temp['visits_saturday'] = x['visits_saturday'][i]
        temp['visits_sunday'] = x['visits_sunday'][i]
        # temp['polygon_wkt'] = x['polygon_wkt'][i]
        temp['latitude'] = x['latitude'][i]
        temp['longitude'] = x['longitude'][i]
        y = y.append(temp)
    y['home_cbg'] = pd.to_numeric(y['home_cbg'], errors='coerce')
    y = y.dropna(subset=['home_cbg'])
    y['home_cbg'] =y['home_cbg'].apply(lambda x: str(x).split('.')[0])
    # print('y', y)
    y = y[y['home_cbg'].isin(county_cbgs)].reset_index(drop = True)
    # print('y_filt', y)
    # y = y.groupby(['date_begin', 'category_tags', 'naics_code', 'distance_from_home', 'includes_parking_lot', 'location_name', 'open_hours', 'raw_visit_counts', 'related_same_day_brand', 'related_same_week_brand', 'sub_category', 'top_category', 'visits_by_day', 'visits_friday','visits_monday', 'visits_saturday', 'visits_sunday', 'visits_thursday', 'visits_tuesday', 'visits_wednesday', 'polygon_wkt', 'home_cbg', 'poi_cbg', 'latitude', 'longitude']).agg({'frequency': sum}).reset_index()
    y = y.groupby(['date_begin', 'category_tags', 'naics_code', 'distance_from_home', 'includes_parking_lot', 'location_name', 'open_hours', 'raw_visit_counts', 'sub_category', 'top_category', 'visits_by_day', 'visits_friday','visits_monday', 'visits_saturday', 'visits_sunday', 'visits_thursday', 'visits_tuesday', 'visits_wednesday', 'home_cbg', 'poi_cbg', 'latitude', 'longitude']).agg({'frequency': sum}).reset_index()
    res = res.append(y)
    res = res.reset_index(drop = True)

In [101]:
res.head()

,date_begin,category_tags,naics_code,distance_from_home,includes_parking_lot,location_name,open_hours,raw_visit_counts,sub_category,top_category,visits_by_day,visits_friday,visits_monday,visits_saturday,visits_sunday,visits_thursday,visits_tuesday,visits_wednesday,home_cbg,poi_cbg,latitude,longitude,frequency
0,2021-01-04,African Food,722513.00,4588.00,false,Angkorians,nan,10.00,Limited-Service Restaurants,Restaurants and Other Eating Places,"[2,3,2,1,0,2,0]",0,2,2,0,1,3,2,470370156261,470370156261,36.07,-86.64,4.00
1,2021-01-04,Churches,813110.00,30258.00,false,Hope Community Church,nan,25.00,Religious Organizations,Religious Organizations,"[4,8,0,7,2,3,1]",2,4,3,1,7,8,0,470370133004,470370133003,36.16,-86.85,4.00
2,2021-01-04,"Churches,Hindu Temple,Mosque,Sikh Temple,Synag...",813110.00,13111.00,false,Sharing Hearts Joining Hands Corp,nan,9.00,Religious Organizations,Religious Organizations,"[0,0,1,1,3,2,2]",3,0,2,2,1,0,1,470370103012,470370189023,36.07,-86.73,4.00
3,2021-01-04,"Country Club,Golf,Golf Lessons,Sports Clubs",713910.00,16136.00,true,Gaylord Springs Golf Links,"{ ""Mon"": [[""7:00"", ""19:00""]], ""Tue"": [[""7:00"",...",26.00,Golf Courses and Country Clubs,Other Amusement and Recreation Industries,"[1,0,8,6,9,1,1]",9,1,1,1,6,0,8,470370133004,470370153002,36.22,-86.68,4.00
4,2021-01-04,nan,722513.00,13519.00,nan,Firehouse Subs,"{ ""Mon"": [[""10:30"", ""21:00""]], ""Tue"": [[""10:30...",82.00,Limited-Service Restaurants,Restaurants and Other Eating Places,"[11,14,11,22,13,8,3]",13,11,8,3,22,14,11,470370108012,470370182011,36.14,-86.88,4.00


In [102]:
county_cbg = pd.read_csv('../data/county_cbg.csv')
county_cbg.GEOID = county_cbg.GEOID.astype(str)
county_cbg.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,47,37,15402,3,470370154023,Block Group 3,G5030,S,1262130,0,36.21,-86.63,"POLYGON ((-86.63448199999999 36.214409, -86.63..."
1,47,37,10702,2,470370107022,Block Group 2,G5030,S,630738,24479,36.25,-86.72,"POLYGON ((-86.723056 36.248187, -86.722477 36...."
2,47,37,10801,3,470370108013,Block Group 3,G5030,S,1839358,0,36.27,-86.73,"POLYGON ((-86.737515 36.264692, -86.737213 36...."
3,47,37,11001,3,470370110013,Block Group 3,G5030,S,1242812,0,36.23,-86.77,"POLYGON ((-86.77301399999999 36.225266, -86.77..."
4,47,37,10904,2,470370109042,Block Group 2,G5030,S,1378206,0,36.23,-86.78,"POLYGON ((-86.78634699999999 36.237884, -86.78..."


In [105]:
import math
t = res.groupby('home_cbg').first().reset_index()
t.home_cbg = t.home_cbg
# .apply(lambda x: str(math.trunc(x)))
t.merge(county_cbg, left_on='home_cbg', right_on='GEOID').to_csv('../data/sg_cbgs.csv', index=False)

In [106]:
import math
t = res.groupby('poi_cbg').first().reset_index()
t.poi_cbg = t.poi_cbg
# .apply(lambda x: str(math.trunc(x)))
t.merge(county_cbg, left_on='poi_cbg', right_on='GEOID').to_csv('../data/sg_poi_cbgs.csv', index=False)

In [107]:
t.head()

,poi_cbg,date_begin,category_tags,naics_code,distance_from_home,includes_parking_lot,location_name,open_hours,raw_visit_counts,sub_category,top_category,visits_by_day,visits_friday,visits_monday,visits_saturday,visits_sunday,visits_thursday,visits_tuesday,visits_wednesday,home_cbg,latitude,longitude,frequency
0,470370111002,2021-03-08,nan,453110.00,10105.00,false,Ann Smith's Florist,"{ ""Mon"": [[""8:00"", ""17:00""]], ""Tue"": [[""8:00"",...",17.00,Florists,Florists,"[6,0,4,3,2,2,0]",2,6,2,0,3,0,4,470370108011,36.23,-86.73,4.00
1,470370122001,2021-02-08,"Gourmet,Meats",445210.00,9370.00,false,Roy's Meat Service,"{ ""Mon"": [[""6:00"", ""20:00""]], ""Tue"": [[""6:00"",...",23.00,Meat Markets,Specialty Food Stores,"[2,3,4,2,6,4,2]",6,2,4,2,2,3,4,470370122001,36.17,-86.74,5.00
2,470370127013,2021-02-22,Churches,813110.00,5538.00,false,Faith United Missionary Baptist Church,nan,24.00,Religious Organizations,Religious Organizations,"[2,10,1,3,3,2,3]",3,2,2,3,3,10,1,470370128023,36.22,-86.81,6.00
3,470370133003,2021-01-04,Churches,813110.00,30258.00,false,Hope Community Church,nan,25.00,Religious Organizations,Religious Organizations,"[4,8,0,7,2,3,1]",2,4,3,1,7,8,0,470370133004,36.16,-86.85,4.00
4,470370144001,2021-03-01,nan,621493.00,36152.00,false,Concentra,"{ ""Mon"": [[""7:00"", ""17:00""]], ""Tue"": [[""7:00"",...",26.00,Freestanding Ambulatory Surgical and Emergency...,Outpatient Care Centers,"[8,1,5,6,6,0,0]",6,8,0,0,6,1,5,470370134002,36.16,-86.79,4.00


In [53]:
# county_cbg = gpd.read_file('../data/Tennessee Census Block Groups/tl_2020_47_bg.shp')

In [108]:
from datetime import datetime
# start_date = datetime.strptime('Jan 11 2021  11:00PM', '%b %d %Y %I:%M%p').date()
# end_date = datetime.strptime('Mar 16 2021  12:00AM', '%b %d %Y %I:%M%p').date()
# # end_date
exact_date = datetime.strptime('Jan 11 2021  12:00AM', '%b %d %Y %I:%M%p').date()
res2 = res
res2.home_cbg = res2.home_cbg.astype(str)
county_cbg.GEOID = county_cbg.GEOID.astype(str)
# res2.home_cbg = res2.home_cbg.astype('double') 
# res2.home_cbg = res2.home_cbg.astype('int64') 
# res2.home_cbg = res2.home_cbg.astype('str') 

# res2[(res2.date_begin>=start_date) & (res2.date_begin<=end_date)].reset_index(drop=True)
# res2[res2.date_begin==exact_date].reset_index(drop=True)
res3 = res2[res2.date_begin==exact_date].merge(county_cbg[['TRACTCE', 'GEOID', 'NAMELSAD']], left_on='home_cbg', right_on='GEOID').drop('GEOID', axis=1)

# res3.to_csv('ham_sg_jan_week2_2021.csv', index=False)

In [109]:
# res3[res3.date_begin==exact_date]

In [110]:
res2.to_csv('../data/county_sg_jan_mar_21_reduced_cols.csv')
res3.to_csv('../data/county_sg_week2_jan21_reduced_cols.csv')

In [62]:
# for key in t[1].groups:
#     x = t[1].get_group(key)
#     x = x.reset_index(drop = True)
#     y = pd.DataFrame()
#     for i in range(len(x)):
#         temp = x['visitor_info'][i]
#         temp['date_begin'] = x['date_begin'][i]
#         temp['poi_cbg'] = x['poi_cbg'][i]
#         temp['category_tags'] = x['category_tags'][i]
#         temp['distance_from_home'] = x['distance_from_home'][i]
#         temp['includes_parking_lot'] = x['includes_parking_lot'][i]
#         temp['location_name'] = x['location_name'][i]
#         temp['open_hours'] = x['open_hours'][i]
#         temp['raw_visit_counts'] = x['raw_visit_counts'][i]
#         temp['related_same_day_brand'] = x['related_same_day_brand'][i]
#         temp['related_same_week_brand'] = x['related_same_week_brand'][i]
#         temp['sub_category'] = x['sub_category'][i]
#         temp['top_category'] = x['top_category'][i]
#         temp['visits_by_day'] = x['visits_by_day'][i]
#         temp['visits_monday'] = x['visits_monday'][i]
#         temp['visits_tuesday'] = x['visits_tuesday'][i]
#         temp['visits_wednesday'] = x['visits_wednesday'][i]
#         temp['visits_thursday'] = x['visits_thursday'][i]
#         temp['visits_friday'] = x['visits_friday'][i]
#         temp['visits_saturday'] = x['visits_saturday'][i]
#         temp['visits_sunday'] = x['visits_sunday'][i]
#         temp['polygon_wkt'] = x['polygon_wkt'][i]
#         y = y.append(temp)
#     y['home_cbg'] = pd.to_numeric(y['home_cbg'], errors='coerce')
#     y = y.dropna(subset=['home_cbg'])
#     y['home_cbg'] =y['home_cbg'].astype(float)
#     y = y[y['home_cbg'].isin(county_cbgs)].reset_index(drop = True)
#     y = y.groupby(['date_begin', 'category_tags', 'home_cbg', 'poi_cbg']).agg({'frequency': sum}).reset_index()
#     res = res.append(y)
#     res = res.reset_index(drop = True)

In [66]:
# for key in t[2].groups:
#     x = t[2].get_group(key)
#     x = x.reset_index(drop = True)
#     y = pd.DataFrame()
#     for i in range(len(x)):
#         temp = x['visitor_info'][i]
#         temp['date_begin'] = x['date_begin'][i]
#         temp['poi_cbg'] = x['poi_cbg'][i]
#         # temp['brands'] = x['brands'][i]
#         temp['category_tags'] = x['category_tags'][i]
#         temp['distance_from_home'] = x['distance_from_home'][i]
#         temp['includes_parking_lot'] = x['includes_parking_lot'][i]
#         temp['location_name'] = x['location_name'][i]
#         temp['open_hours'] = x['open_hours'][i]
#         temp['raw_visit_counts'] = x['raw_visit_counts'][i]
#         temp['related_same_day_brand'] = x['related_same_day_brand'][i]
#         temp['related_same_week_brand'] = x['related_same_week_brand'][i]
#         temp['sub_category'] = x['sub_category'][i]
#         temp['top_category'] = x['top_category'][i]
#         temp['visits_by_day'] = x['visits_by_day'][i]
#         temp['visits_monday'] = x['visits_monday'][i]
#         temp['visits_tuesday'] = x['visits_tuesday'][i]
#         temp['visits_wednesday'] = x['visits_wednesday'][i]
#         temp['visits_thursday'] = x['visits_thursday'][i]
#         temp['visits_friday'] = x['visits_friday'][i]
#         temp['visits_saturday'] = x['visits_saturday'][i]
#         temp['visits_sunday'] = x['visits_sunday'][i]
#         temp['polygon_wkt'] = x['polygon_wkt'][i]
#         y = y.append(temp)
#     y['home_cbg'] = pd.to_numeric(y['home_cbg'], errors='coerce')
#     y = y.dropna(subset=['home_cbg'])
#     y['home_cbg'] =y['home_cbg'].astype(float)
#     y = y[y['home_cbg'].isin(county_cbgs)].reset_index(drop = True)
#     y = y.groupby(['date_begin', 'category_tags', 'distance_from_home', 'includes_parking_lot', 'location_name', 'open_hours', 'raw_visit_counts', 'related_same_day_brand', 'related_same_week_brand', 'sub_category', 'top_category', 'visits_by_day', 'visits_friday','visits_monday', 'visits_saturday', 'visits_sunday', 'visits_thursday', 'visits_tuesday', 'visits_wednesday', 'polygon_wkt', 'home_cbg', 'poi_cbg']).agg({'frequency': sum}).reset_index()
#     res = res.append(y)
#     res = res.reset_index(drop = True)

In [68]:
# for key in t[3].groups:
#     x = t[3].get_group(key)
#     x = x.reset_index(drop = True)
#     y = pd.DataFrame()
#     for i in range(len(x)):
#         temp = x['visitor_info'][i]
#         temp['date_begin'] = x['date_begin'][i]
#         temp['poi_cbg'] = x['poi_cbg'][i]
#         temp['category_tags'] = x['category_tags'][i]
#         temp['distance_from_home'] = x['distance_from_home'][i]
#         temp['includes_parking_lot'] = x['includes_parking_lot'][i]
#         temp['location_name'] = x['location_name'][i]
#         temp['open_hours'] = x['open_hours'][i]
#         temp['raw_visit_counts'] = x['raw_visit_counts'][i]
#         temp['related_same_day_brand'] = x['related_same_day_brand'][i]
#         temp['related_same_week_brand'] = x['related_same_week_brand'][i]
#         temp['sub_category'] = x['sub_category'][i]
#         temp['top_category'] = x['top_category'][i]
#         temp['visits_by_day'] = x['visits_by_day'][i]
#         temp['visits_monday'] = x['visits_monday'][i]
#         temp['visits_tuesday'] = x['visits_tuesday'][i]
#         temp['visits_wednesday'] = x['visits_wednesday'][i]
#         temp['visits_thursday'] = x['visits_thursday'][i]
#         temp['visits_friday'] = x['visits_friday'][i]
#         temp['visits_saturday'] = x['visits_saturday'][i]
#         temp['visits_sunday'] = x['visits_sunday'][i]
#         temp['polygon_wkt'] = x['polygon_wkt'][i]
#         y = y.append(temp)
#     y['home_cbg'] = pd.to_numeric(y['home_cbg'], errors='coerce')
#     y = y.dropna(subset=['home_cbg'])
#     y['home_cbg'] =y['home_cbg'].astype(float)
#     y = y[y['home_cbg'].isin(county_cbgs)].reset_index(drop = True)
#     y = y.groupby(['date_begin', 'category_tags', 'distance_from_home', 'includes_parking_lot', 'location_name', 'open_hours', 'raw_visit_counts', 'related_same_day_brand', 'related_same_week_brand', 'sub_category', 'top_category', 'visits_by_day', 'visits_friday','visits_monday', 'visits_saturday', 'visits_sunday', 'visits_thursday', 'visits_tuesday', 'visits_wednesday', 'polygon_wkt', 'home_cbg', 'poi_cbg']).agg({'frequency': sum}).reset_index()
#     res = res.append(y)
#     res = res.reset_index(drop = True)

In [69]:
# for key in t[4].groups:
#     x = t[4].get_group(key)
#     x = x.reset_index(drop = True)
#     y = pd.DataFrame()
#     for i in range(len(x)):
#         temp = x['visitor_info'][i]
#         temp['date_begin'] = x['date_begin'][i]
#         temp['poi_cbg'] = x['poi_cbg'][i]
#         # temp['brands'] = x['brands'][i]
#         temp['category_tags'] = x['category_tags'][i]
#         temp['distance_from_home'] = x['distance_from_home'][i]
#         temp['includes_parking_lot'] = x['includes_parking_lot'][i]
#         temp['location_name'] = x['location_name'][i]
#         temp['open_hours'] = x['open_hours'][i]
#         temp['raw_visit_counts'] = x['raw_visit_counts'][i]
#         temp['related_same_day_brand'] = x['related_same_day_brand'][i]
#         temp['related_same_week_brand'] = x['related_same_week_brand'][i]
#         temp['sub_category'] = x['sub_category'][i]
#         temp['top_category'] = x['top_category'][i]
#         temp['visits_by_day'] = x['visits_by_day'][i]
#         temp['visits_monday'] = x['visits_monday'][i]
#         temp['visits_tuesday'] = x['visits_tuesday'][i]
#         temp['visits_wednesday'] = x['visits_wednesday'][i]
#         temp['visits_thursday'] = x['visits_thursday'][i]
#         temp['visits_friday'] = x['visits_friday'][i]
#         temp['visits_saturday'] = x['visits_saturday'][i]
#         temp['visits_sunday'] = x['visits_sunday'][i]
#         temp['polygon_wkt'] = x['polygon_wkt'][i]
#         y = y.append(temp)
#     y['home_cbg'] = pd.to_numeric(y['home_cbg'], errors='coerce')
#     y = y.dropna(subset=['home_cbg'])
#     y['home_cbg'] =y['home_cbg'].astype(float)
#     y = y[y['home_cbg'].isin(county_cbgs)].reset_index(drop = True)
#     y = y.groupby(['date_begin', 'category_tags', 'distance_from_home', 'includes_parking_lot', 'location_name', 'open_hours', 'raw_visit_counts', 'related_same_day_brand', 'related_same_week_brand', 'sub_category', 'top_category', 'visits_by_day', 'visits_friday','visits_monday', 'visits_saturday', 'visits_sunday', 'visits_thursday', 'visits_tuesday', 'visits_wednesday', 'polygon_wkt', 'home_cbg', 'poi_cbg']).agg({'frequency': sum}).reset_index()
#     res = res.append(y)
#     res = res.reset_index(drop = True)

In [71]:
# res.to_csv("county_allweeks2021_new.csv")

In [105]:
# res2 = res
# res2.home_cbg = res2.home_cbg.astype('double') 
# res2.home_cbg = res2.home_cbg.astype('int64') 
# res2.home_cbg = res2.home_cbg.astype('str') 

In [130]:
# res3 = res2.merge(county_cbg[['TRACTCE', 'GEOID', 'NAMELSAD', 'geometry']], left_on='home_cbg', right_on='GEOID').drop('GEOID', axis=1)

In [132]:
# res3.to_csv('county_allweeks2021_cbg_names.csv', index=False)

In [136]:
# res3[res3.date_begin<'2021-02-01']

,date_begin,category_tags,distance_from_home,includes_parking_lot,location_name,open_hours,raw_visit_counts,related_same_day_brand,related_same_week_brand,sub_category,top_category,visits_by_day,visits_friday,visits_monday,visits_saturday,visits_sunday,visits_thursday,visits_tuesday,visits_wednesday,polygon_wkt,home_cbg,poi_cbg,frequency,TRACTCE,NAMELSAD,geometry
0,2021-01-04,"3D Printing,Carpenters,Electronics,Fabrics,Ind...",10016.00,True,Amt,NaN,145.00,"{""Mapco"":7,""Starbucks"":7,""Walmart"":6,""RaceWay""...","{""Walmart"":23,""Starbucks"":17,""Mapco"":15,""Circl...",NaN,Other Miscellaneous Manufacturing,"[26,22,28,26,30,9,4]",30.00,26.00,9.00,4.00,26.00,22.00,28.00,POLYGON ((-85.32268357242836 35.04184258851209...,470650006001,470650016001.00,7.00,000600,Block Group 1,"POLYGON ((-85.30907 35.06679, -85.30905 35.067..."
1,2021-01-04,"Arts and Crafts,Men's Clothing,Children's Clot...",8323.00,False,Walmart Supercenter,NaN,1212.00,"{""Murphy USA"":4,""RaceWay"":4,""Starbucks"":3,""Foo...","{""McDonald's"":16,""Dollar General"":14,""Starbuck...",Warehouse Clubs and Supercenters,"General Merchandise Stores, including Warehous...","[161,177,146,147,165,198,218]",165.00,161.00,198.00,218.00,147.00,177.00,146.00,"POLYGON ((-85.33188589552984 35.0935382630435,...",470650006001,470650109021.00,4.00,000600,Block Group 1,"POLYGON ((-85.30907 35.06679, -85.30905 35.067..."
2,2021-01-04,"Bar or Pub,Cocktail Lounge,Late Night,Mexican ...",12204.00,False,Mezcla Cocina Y Cantina,NaN,60.00,"{""Walmart"":9,""Walgreens"":7,""McDonald's"":7,""Exx...","{""Walmart"":31,""McDonald's"":20,""Starbucks"":17,""...",Full-Service Restaurants,Restaurants and Other Eating Places,"[6,16,9,6,5,14,4]",5.00,6.00,14.00,4.00,6.00,16.00,9.00,POLYGON ((-85.31718255999994 35.06874119000002...,470650006001,470650008001.00,4.00,000600,Block Group 1,"POLYGON ((-85.30907 35.06679, -85.30905 35.067..."
3,2021-01-04,"Building Supplies,Flooring,Appliances,Hardware...",11218.00,False,Lowe's,"{ ""Mon"": [[""6:00"", ""21:00""]], ""Tue"": [[""6:00"",...",755.00,"{""Walmart"":11,""The Home Depot"":7,""Circle K"":5,...","{""Walmart"":40,""The Home Depot"":16,""Circle K"":1...",Hardware Stores,Building Material and Supplies Dealers,"[111,89,96,90,115,129,125]",115.00,111.00,129.00,125.00,90.00,89.00,96.00,POLYGON ((-85.24546929599995 35.13851526400003...,470650006001,470650104314.00,4.00,000600,Block Group 1,"POLYGON ((-85.30907 35.06679, -85.30905 35.067..."
4,2021-01-04,Car Wash,4707.00,False,Exquisite Hand Car Wash & Detail Shop II,NaN,16.00,"{""Cook Out"":19,""Popeyes Louisiana Kitchen"":19,...","{""Walmart"":33,""Mapco"":33,""McDonald's"":33,""Taco...",NaN,Automotive Repair and Maintenance,"[0,7,2,0,5,2,0]",5.00,0.00,2.00,0.00,0.00,7.00,2.00,POLYGON ((-85.26983710701583 35.03372682681248...,470650006001,470650013001.00,4.00,000600,Block Group 1,"POLYGON ((-85.30907 35.06679, -85.30905 35.067..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009006,2021-01-04,Industrial,30519.00,True,Chattanooga Metropolitan Airport Solar,NaN,962.00,"{""Walmart"":4,""Chick-fil-A"":4,""McDonald's"":4,""S...","{""Walmart"":28,""McDonald's"":16,""Chick-fil-A"":13...",Solar Electric Power Generation,"Electric Power Generation, Transmission and Di...","[183,128,140,143,140,109,119]",140.00,183.00,109.00,119.00,143.00,128.00,140.00,POLYGON ((-85.20947304020339 35.04524622127125...,470659801001,470659801001.00,5.00,980100,Block Group 1,"POLYGON ((-85.21402 35.04923, -85.21395 35.049..."
1009007,2021-01-11,Industrial,39137.00,True,Chattanooga Metropolitan Airport Solar,NaN,961.00,"{""Walmart"":7,""Chick-fil-A"":4,""McDonald's"":4,""P...","{""Walmart"":27,""McDonald's"":14,""Starbucks"":12,""...",Solar Electric Power Generation,"Electric Power Generation, Transmission and Di...","[132,130,146,161,158,111,123]",158.00,132.00,111.00,123.00,161.00,130.00,146.00,POLYGON ((-85.20947304020339 35.04524622127125...,470659801001,470659801001.0

In [127]:
# pd.set_option('display.max_columns', None)
# location_df = pd.DataFrame(res3.location_name.unique(), columns=['locations']).sort_values('locations')
# # .to_csv('locations.csv', index=False)
# location_df.merge(res3[['location_name', 'category_tags']], left_on='locations', right_on='location_name', how='left')


,locations,location_name,category_tags
0,1885 Grill,1885 Grill,"American Food,Brunch,Seafood"
1,1885 Grill,1885 Grill,"American Food,Brunch,Seafood"
2,1885 Grill,1885 Grill,"American Food,Brunch,Seafood"
3,1885 Grill,1885 Grill,"American Food,Brunch,Seafood"
4,1885 Grill,1885 Grill,"American Food,Brunch,Seafood"
...,...,...,...
1009054,NaN,NaN,"Jewelry,Jewelry Repair"
1009055,NaN,NaN,Trophy Shop
1009056,NaN,NaN,Industrial
1009057,NaN,NaN,NaN


In [33]:
# res['category_tags'].value_counts().nlargest(20)

nan                                                                                                         215191
Arts and Crafts,Men's Clothing,Children's Clothing,Home Décor,Baby Supplies                                  29054
Counter Service,Lunch,Fast Food,Drive Through,Breakfast,Dinner,Burgers                                       27968
Medical Supplies                                                                                             14504
Dialysis,Hospice,Physical Therapy,Walk-in Clinic                                                             11372
Mexican Food                                                                                                 11289
Snacks,Counter Service,Dessert,Tea House,Coffee Shop,Bakery                                                  10875
Commercial Real Estate,Property Management                                                                   10189
Counter Service,Late Night,Lunch,Fast Food,Drive Through,Breakfast,Dinner,Burger

In [45]:
# sample = safe_df.sample(n = 50)
# sample.to_csv("sample.csv")